# Portfolio Review (90 Days) – Notebook

**Goal:** Assess revenue, loyalty, and risk signals by region/segment. Prepare 3–4 charts and a 200–300 word summary.

In [1]:
import pandas as pd
from datetime import datetime, timedelta

DATA_DIR = '../'
customers = pd.read_csv(DATA_DIR + 'customers.csv', parse_dates=['join_date'])
accounts = pd.read_csv(DATA_DIR + 'accounts.csv', parse_dates=['open_date'])
transactions = pd.read_csv(DATA_DIR + 'transactions.csv', parse_dates=['txn_date'])
chargebacks = pd.read_csv(DATA_DIR + 'chargebacks.csv', parse_dates=['cb_date'])
balances = pd.read_csv(DATA_DIR + 'monthly_balances.csv')
churn = pd.read_csv(DATA_DIR + 'customer_churn.csv', parse_dates=['last_txn_date'])

today = datetime(2025,10,10)
cut90 = today - timedelta(days=90)
cut30 = today - timedelta(days=30)

# Join helpers
cust_acct = accounts.merge(customers, on='customer_id', how='left')
txn_full = transactions.merge(accounts[['account_id','customer_id','account_type']], on='account_id', how='left')\
                     .merge(customers[['customer_id','region','segment','age']], on='customer_id', how='left')

# Save joined tables for future use
cust_acct.to_csv('../cust_acct.csv', index = False)
txn_full.to_csv('../txn_full.csv', index = False)
cust_acct.head(), txn_full.head()


(  account_id customer_id account_type  open_date  join_date region  age  \
 0    A000001      C00001      Current 2025-01-31 2025-01-31  North   46   
 1    A000002      C00001      Savings 2025-01-31 2025-01-31  North   46   
 2    A000003      C00001   CreditCard 2025-01-31 2025-01-31  North   46   
 3    A000004      C00002      Savings 2024-05-10 2024-05-10  North   31   
 4    A000005      C00003      Current 2025-02-04 2025-02-04  South   32   
 
    annual_income  segment  tenure_days  
 0       34879.27  Premier          252  
 1       34879.27  Premier          252  
 2       34879.27  Premier          252  
 3       29433.18      SME          518  
 4       48751.53   Retail          248  ,
   transaction_id account_id   txn_date category channel   amount customer_id  \
 0       T0000001    A000001 2025-02-18      ATM  Mobile   -44.06      C00001   
 1       T0000002    A000001 2025-02-15   Online     ATM   -47.39      C00001   
 2       T0000003    A000001 2025-07-25      F

## A) KPIs & Definitions
1) Revenue last 90 days by region/segment.  
2) Active customers (≥1 txn last 30d); Loyal (tenure ≥365d).  
3) Top 10% customers by revenue - analysis  
4) Refund analysis - monthly statistics.  
5) Avg monthly ending balance.


In [129]:
# 1) Revenue last 90 days
# Filter by date
today = pd.Timestamp.now()
days_ago_90 = today - pd.Timedelta(days = 90)
ninety_days_mask = txn_full['txn_date'].between(days_ago_90, today)
ninety_days_filter = txn_full[ninety_days_mask]
# Get revenue (generated from fees and interest)
revenue_categories = ['Fee', 'Interest']
revenue_mask = ninety_days_filter['category'].isin(revenue_categories)
revenue = ninety_days_filter[revenue_mask]
# Group by region and segment + basic metrics
grouped_revenue = revenue.groupby(by = ['region', 'segment', 'category']).agg({'amount': ['sum', 'mean', 'count']}, {'transaction_id': 'count'})
print('Revenue in the last 90 days:')
grouped_revenue.head(10)

Revenue in the last 90 days:


amount                 
                            sum       mean count
region segment category                         
East   Premier Fee      -140.00 -15.555556     9
               Interest   19.50   2.437500     8
       Retail  Fee      -555.00 -13.536585    41
               Interest  122.72   2.611064    47
       SME     Fee       -25.00  -8.333333     3
               Interest    5.92   1.973333     3
North  Premier Fee      -120.00 -12.000000    10
               Interest   25.92   2.592000    10
       Retail  Fee      -740.00 -11.746032    63
               Interest   93.52   2.280976    41

In [ ]:
# 2.1) Active customers
days_ago_30 = today - pd.Timedelta(days = 30)
thirty_days_mask = txn_full['txn_date'].between(days_ago_30, today)
thirty_days_txn  = txn_full[thirty_days_mask]
active_customer = thirty_days_txn['account_id'].unique()
print(f'Number of active customers: {len(active_customer)}')

# 2.2) Loyal customers
loyalty_mask = cust_acct['tenure_days'] >= 365
loyal_tenure = cust_acct[loyalty_mask]
loyal_customers = loyal_tenure['customer_id'].unique()
print(f'Number of loyal customers: {len(loyal_customers)}')

Number of active customers: 787
Number of loyal customers: 664


In [91]:
# 3) Top 10% customers by revenue (90d) - grouped by segment
revenue_df = revenue.copy()
revenue_df.loc[:, 'amount'] = revenue_df['amount'].abs()
customers_sorted = revenue_df.groupby(by = 'customer_id')['amount'].sum().sort_values(ascending = False)
threshold = customers_sorted.quantile(0.9)
top_customers = customers_sorted[customers_sorted > threshold]
print(f'Number of top customers (10%): {len(top_customers)}')
print(f'Number of customers: {len(customers_sorted)}')
print(f'Value threshold: £{threshold:,.2f}')
print(f'Total value from top 10%: {top_customers.sum():,.2f}')

Number of top customers (10%): 29
Number of customers: 283
Value threshold: £28.39
Total value from top 10%: 1,077.93


In [128]:
# 3) Top customer profile
top_customer_profile = revenue_df[revenue_df['customer_id'].isin(top_customers.index)].groupby(['segment', 'region', 'category'])['customer_id'].nunique().unstack(fill_value = 0)
print('Top Customer Profile:')
top_customer_profile.head(10)

Top Customer Profile:


category        Fee  Interest
segment region               
Premier East      2         1
        North     2         1
        West      2         0
Retail  East      6         4
        North     8         5
        South     3         2
        West      6         5

In [ ]:
# 4) Refund analysis
# Filter relevant transactions (POS, Online) and Refunds
card_txns_mask = txn_full['category'].isin(['POS', 'Online'])
refund_mask = txn_full['category'] == 'Refund'

# Monthly stats
monthly_refund_stats = txn_full.groupby(pd.Grouper(key = 'txn_date', freq = 'ME')).agg


SpecificationError: nested renamer is not supported

## B) Visuals
Create and save 3–4 simple charts (matplotlib): revenue by region×loyalty, churn by tenure bucket, and chargeback rate trend, regional breakdown of top customers, average transaction sizes between groups


In [ ]:
import matplotlib.pyplot as plt

# Revenue by loyalty & region



## C) Write-up
Draft a 200–300 word exec summary with 3 prioritised actions and expected impact.
